In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Chapter4Project/NeedToFix

Mounted at /content/drive
/content/drive/MyDrive/Chapter4Project/NeedToFix


/content/drive/MyDrive/Chapter4Project/NeedToFix


In [2]:
!pip install accelerate
!pip install git+https://github.com/huggingface/diffusers@v0.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/diffusers (to revision v0.20.0) to /tmp/pip-req-build-ldo5blhl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-ldo5blhl
  Running command git checkout -q 58732ec0df75f1dbc0a2c12ac0b90c7d675288fc
  Resolved https://github.com/huggingface/diffusers to commit 58732ec0df75f1dbc0a2c12ac0b90c7d675288fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.20.0-py3-none-any.whl size=1342635 sha256=91424846d08622e8b7e213e690a839d730ed6e8f1f90e5a9a04741ee1ed3bfd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8zgal1en/

In [3]:
#cmd for obtaining/saving feature files for each image
!python features.py --source ../Target_Img/NewTarget --save ./featureRAW

2023-09-09 00:13:22.698230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 00:13:23.755717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/drive/MyDrive/Chapter4Project/NeedToFix/SR_pipe.py:19: FutureWarning: Importing `DiffusionPipeline` or `ImagePipelineOutput` from diffusers.pipeline_utils is deprecated. Please import from diffusers.pipelines.pipeline_utils instead.
  from diffusers.pipeline_utils import DiffusionPipeline, ImagePipelineOutput
vqvae/diffusion_pytorch_model.safetensors not found
Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]
Fetching 6 files:  33% 2/6 [00:00<00:01,  3.00it/s]

Fetching 6 files:  50% 3/6 [00:00<00:00,  3.95it/s]










Fetching 6 files: 100% 6/

In [3]:
#cmd for combining .pt files
!python combine.py --source ./combine --save ./combine2

Combined 2 .pt files to a torch.Size([6497, 49152]) tensor



In [5]:
#Matching cmd
!python match_diffSR.py --source ./match_img --target-feat ./features.pt --target ../Target_Img --save ./match_results/RAWdot --topk 15

2023-09-09 04:31:55.282752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 04:31:56.366083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/drive/MyDrive/Chapter4Project/NeedToFix/SR_pipe.py:19: FutureWarning: Importing `DiffusionPipeline` or `ImagePipelineOutput` from diffusers.pipeline_utils is deprecated. Please import from diffusers.pipelines.pipeline_utils instead.
  from diffusers.pipeline_utils import DiffusionPipeline, ImagePipelineOutput
unet/diffusion_pytorch_model.safetensors not found
Loading pipeline components...: 100% 3/3 [00:00<00:00,  3.61it/s]
100% 100/100 [00:04<00:00, 22.31it/s]
100% 100/100 [00:02<00:00, 40.91it/s]
100% 100/100 [00:02<00:00, 40.64it/s]
100% 100/100 

Ignore Below

In [ ]:
%ls

In [ ]:
import requests
from PIL import Image
from io import BytesIO
# from diffusers.src.diffusers.pipelines.latent_diffusion import LDMSuperResolutionPipeline
from SR_pipe import LDMSuperResolutionPipeline
import torch
import os
import shutil
import numpy as np

In [ ]:
device = "cuda"
model_id = "CompVis/ldm-super-resolution-4x-openimages"

# load model and scheduler
pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id)
pipeline = pipeline.to(device)



In [ ]:
# load img
def load_img(path):
  low_res_img = Image.open(path).convert("RGB")
  size = (128,128)
  low_res_img = low_res_img.resize(size)
  return low_res_img, size


def get_feature_vector(latent_in):
  pool = torch.nn.MaxPool3d((3,4,4), padding=(0,2,2), stride=3)
  output = pool(latent_in)
  fv = torch.flatten(output)
  return fv

def get_features(model, img_path, num_step=100):
  features = []
  names = []
  for fRoot, fDirs, fFiles in os.walk(img_path):
    for ffile in fFiles:

      full_path = os.path.join(fRoot, ffile)
      names.append(full_path)

      low_res, low_res_size = load_img(full_path)
      upscaled, latent_emb = model(low_res, num_inference_steps=num_step, eta=1)
      emb = get_feature_vector(latent_emb)
      features.append(emb.detach().cpu())

  features = torch.stack(features)
  return features, names



def similarity(fv1, fv2):
  return torch.cdist(fv1, fv2)

def save_features(model, path, sv_path, num_step=100):
  features, names = get_features(model, path, num_step)
  torch.save(features, sv_path + '/features.pt')

def run(model, source_path, target_path, sv_path, feat_sv_path, topk, num_step=100):
  source_features, source_name = get_features(model, source_path, num_step)
  target_features, target_name = torch.load(target_path),
  source_features = torch.tensor(source_features)
  target_features = torch.tensor(target_features)
  torch.save(target_features, feat_sv_path + '/target_features.pt')

  sim = similarity(source_features, target_features)

  max_score, max_arg = torch.topk(sim, topk, dim=1, largest=False)

  for i, ip in enumerate(max_arg):
    si_name = source_name[i].split('/')[-1]

    os.makedirs('{}/{}'.format(sv_path, i))
    shutil.copyfile(source_name[i], '{}/{}/SRC_{}'.format(sv_path, i, si_name))

    t_names = []
    for k in range(topk):
      ti_name = target_name[ip[k]].split('/')[-1]
      t_names.append(ti_name)

      shutil.copyfile(target_name[ip[k]], '{}/{}/{}'.format(sv_path, i, ti_name))



In [ ]:
source_img_path = '../Chapter4Project/source_test'
target_img_path = '../Chapter4Project/Target_Img/target_img_features.pt'
img_save_folder = '../Chapter4Project/Matching/outputs/diff_match'

In [ ]:
import os
def get_img_paths(folder_path, mode):
  names = []
  for fRoot, fDirs, fFiles in os.walk(folder_path):
    for ffile in fFiles:
      full_path = fRoot + '/' + ffile
      names.append(full_path)
      if mode == 'print':
        print(full_path)
  if mode == 'count':
    print(len(names))
  return names


In [ ]:
a = get_img_paths('./sv_noPool', 'count')
a

4390


['./sv_noPool/2302.pt',
 './sv_noPool/2303.pt',
 './sv_noPool/2304.pt',
 './sv_noPool/2305.pt',
 './sv_noPool/2306.pt',
 './sv_noPool/2307.pt',
 './sv_noPool/2308.pt',
 './sv_noPool/2309.pt',
 './sv_noPool/2310.pt',
 './sv_noPool/2311.pt',
 './sv_noPool/2312.pt',
 './sv_noPool/2313.pt',
 './sv_noPool/2314.pt',
 './sv_noPool/2315.pt',
 './sv_noPool/2316.pt',
 './sv_noPool/2317.pt',
 './sv_noPool/2318.pt',
 './sv_noPool/2319.pt',
 './sv_noPool/2320.pt',
 './sv_noPool/2321.pt',
 './sv_noPool/2322.pt',
 './sv_noPool/2323.pt',
 './sv_noPool/2324.pt',
 './sv_noPool/2325.pt',
 './sv_noPool/2326.pt',
 './sv_noPool/2327.pt',
 './sv_noPool/2328.pt',
 './sv_noPool/2329.pt',
 './sv_noPool/2330.pt',
 './sv_noPool/2331.pt',
 './sv_noPool/2332.pt',
 './sv_noPool/2333.pt',
 './sv_noPool/2334.pt',
 './sv_noPool/2335.pt',
 './sv_noPool/2336.pt',
 './sv_noPool/2337.pt',
 './sv_noPool/2338.pt',
 './sv_noPool/2339.pt',
 './sv_noPool/2340.pt',
 './sv_noPool/2341.pt',
 './sv_noPool/2342.pt',
 './sv_noPool/23

In [ ]:
os.path.exists('./sv_noPool/3301.pt')

True

In [ ]:
#run matching
up = run(pipeline, source_img_path, target_img_path, img_save_folder,'.', 5)

In [ ]:
#run save features to a pt file
save_features(pipeline, target_img_path, '.', 100)

In [ ]:
print(torch.min(up))

tensor(-3.6429, device='cuda:0')


In [ ]:
import os
def clean_folder(sv_path):
  for img_path in os.listdir(sv_path):
    os.remove(sv_path + '/' + img_path)

#clean_folder('./upscale')

In [ ]:
import time
start_time = time.time()
!python match_diffSR.py --source ../Chapter4Project/source_test --target ../Chapter4Project/target_test --save ../Chapter4Project/Matching/outputs/diff_match --topk 5
print("--- %s seconds ---" % (time.time() - start_time))

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
python3: can't open file 'match_diffSR.py': [Errno 107] Transport endpoint is not connected
--- 0.10508012771606445 seconds ---
